In [49]:
# Modified Primal Tableau generic format is:

# con\var |    b        x1        x2        ...        xn        s1        s2        ...        sq        A1        A2        ...        Ar
# ---------------------------------------------------------------------------------------------------------------------------------------------
# con1    |    b1       x11       x11       ...        x1n       0/1/-1    0/1/-1    ...        0/1/-1    0/1       0/1       ...        0/1
# con2    |    b2       x21       x22       ...        x2n       0/1/-1    0/1/-1    ...        0/1/-1    0/1       0/1       ...        0/1
# .       |    .        .         .         ...        .         .         .         ...        .         .         .         ...        .
# .       |    .        .         .         ...        .         .         .         ...        .         .         .         ...        .
# .       |    .        .         .         ...        .         .         .         ...        .         .         .         ...        .
# conm    |    bm       xm1       xm2       ...        xmn       0/1/-1    0/1/-1    ...        0/1/-1    0/1       0/1       ...        0/1
# z       |    z0       -c1       -c2       ...        0         0         0         ...        0         0         0         ...        0
# zA      |    0        0         0         ...        0         0         0         ...        0         -1        -1        ...        -1

In [50]:
import numpy as np

In [51]:
max_branch_and_bound_recursion_depth = 6

In [52]:
# Objective function
# z = 1 * x1 + -1 * x2

In [53]:
# Constraints
# -2 * x1 + 2 * x2 >= 1
# 1 * x1 + 1 * x2 <= 10
# -8 * x1 + 10 * x2 >= 13
# 1 * x1 + 1 * x2 <= 9

# Add slack/surplus variables
# -2 * x1 + 2 * x2 - s1 = 1
# 1 * x1 + 1 * x2 + s2 = 10
# -8 * x1 + 10 * x2 - s3 = 13
# 1 * x1 + 1 * x2 + s4 = 9

# Converts to
# 1 = -2 * x1 + 2 * x2 - s1
# 10 = 1 * x1 + 1 * x2 + s2
# 13 = -8 * x1 + 10 * x2 - s3
# 9 = 1 * x1 + 1 * x2 + s4

In [54]:
# Constraints in tableau format are
# con\var |    b        x1        x2        s1        s2        s3        s4        A1        A2
# -----------------------------------------------------------------------------------------------
# con1    |    1        -2        2         1         0         0         0         1         0
# con2    |    10       1         1         0         1         0         0         0         0
# con3    |    13       -8        10        0         0         1         0         0         1
# con4    |    9        1         1         0         0         0         1         0         0
# z       |    0        -1        1         0         0         0         0         0         0
# zA      |    0        0         0         0         0         0         0         -1        -1

# Read inputs

### Define if this is a maximization or minimization problem

In [55]:
# This function reads in if the objective function is going to be a maximization or minimization problem
def read_objective_function_maximization_or_minimization():
    maximization_problem = np.loadtxt(fname = "inputs/objective_function_maximization.txt", dtype = np.int64, delimiter = '\t')
    
    print("read_objective_function_maximization_or_minimization: maximization_problem = \n{}".format(maximization_problem))
    
    return maximization_problem

### Get number of constraints and variables

In [56]:
# This function reads the number of constraints and variables
def read_number_of_constraints_and_variables():
    number_of_constraints = np.loadtxt(fname = "inputs/number_of_constraints.txt", dtype = np.int64, delimiter = '\t')
    number_of_variables = np.loadtxt(fname = "inputs/number_of_variables.txt", dtype = np.int64, delimiter = '\t')
    
    print("read_number_of_constraints_and_variables: number_of_constraints = \n{}".format(number_of_constraints))
    print("read_number_of_constraints_and_variables: number_of_variables = \n{}".format(number_of_variables))
    
    return number_of_constraints, number_of_variables

### Get variable special requirements

In [57]:
# This function reads and counts variable special requirements like needing to be an integer, etc.
def read_and_count_variable_special_requirements():
    # shape = (number_of_variables,)
    variable_special_requirements = np.loadtxt(fname = "inputs/variable_special_requirements.txt", dtype = np.int64, delimiter = '\t')
    
    print("read_and_count_variable_special_requirements: variable_special_requirements = \n{}".format(variable_special_requirements))

    number_of_variables_required_to_be_standard = np.count_nonzero(a = (variable_special_requirements == 0))
    number_of_variables_required_to_be_integer = np.count_nonzero(a = (variable_special_requirements == 1))
    number_of_variables_required_to_be_binary = np.count_nonzero(a = (variable_special_requirements == 2))
    number_of_variables_required_to_be_unrestricted = np.count_nonzero(a = (variable_special_requirements == 3))

    print("read_and_count_variable_special_requirements: number_of_variables_required_to_be_standard = \n{}".format(number_of_variables_required_to_be_standard))
    print("read_and_count_variable_special_requirements: number_of_variables_required_to_be_integer = \n{}".format(number_of_variables_required_to_be_integer))
    print("read_and_count_variable_special_requirements: number_of_variables_required_to_be_binary = \n{}".format(number_of_variables_required_to_be_binary))
    print("read_and_count_variable_special_requirements: number_of_variables_required_to_be_unrestricted = \n{}".format(number_of_variables_required_to_be_unrestricted))
    
    return variable_special_requirements, number_of_variables_required_to_be_standard, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, number_of_variables_required_to_be_unrestricted

### Create objective function

In [58]:
# This function reads in the objective function's initial constant
def read_objective_function_initial_constant():
    # shape = (2,)
    objective_function_initial_constant_numerator = np.loadtxt(fname = "inputs/objective_function_initial_constant_numerator.txt", dtype = np.int64, delimiter = '\t')
    objective_function_initial_constant_denominator = np.loadtxt(fname = "inputs/objective_function_initial_constant_denominator.txt", dtype = np.int64, delimiter = '\t')
    
    objective_function_initial_constant = np.stack(arrays = [objective_function_initial_constant_numerator, objective_function_initial_constant_denominator], axis = 0)
    
    print("read_objective_function_initial_constant: objective_function_initial_constant = \n{}".format(objective_function_initial_constant[0].astype(np.float64) / objective_function_initial_constant[1]))
    
    return objective_function_initial_constant

In [59]:
# This function reads the objective functions variable coefficients
def read_objective_function_coefficient_vector():
    # shape = (2, number_of_variables)
    objective_function_coefficient_vector_numerator = np.loadtxt(fname = "inputs/objective_function_coefficient_vector_numerator.txt", dtype = np.int64, delimiter = '\t')
    objective_function_coefficient_vector_denominator = np.loadtxt(fname = "inputs/objective_function_coefficient_vector_denominator.txt", dtype = np.int64, delimiter = '\t')
    
    objective_function_coefficient_vector = np.stack(arrays = [objective_function_coefficient_vector_numerator, objective_function_coefficient_vector_denominator], axis = 0)
    
    print("read_objective_function_coefficient_vector: objective_function_coefficient_vector = \n{}".format(objective_function_coefficient_vector[0].astype(np.float64) / objective_function_coefficient_vector[1]))
    
    return objective_function_coefficient_vector

### Get initial constraint inequality directions (<= (1), >= (-1), = (0))

In [60]:
# This function reads the constraint inequality directions
def read_constraint_inequality_direction_vector():
    # shape = (number_of_constraints,)
    constraint_inequality_direction_vector = np.loadtxt(fname = "inputs/constraint_inequality_direction_vector.txt", dtype = np.int64, delimiter = '\t')
    
    print("read_constraint_inequality_direction_vector: constraint_inequality_direction_vector = \n{}".format(constraint_inequality_direction_vector))
    
    return constraint_inequality_direction_vector

### Get constraint constants

In [61]:
# This function reads the constraint constants
def read_constraint_constant_vector():
    # shape = (number_of_constraints,)
    constraint_constant_vector_numerator = np.loadtxt(fname = "inputs/constraint_constant_vector_numerator.txt", dtype = np.int64, delimiter = '\t')
    constraint_constant_vector_denominator = np.loadtxt(fname = "inputs/constraint_constant_vector_denominator.txt", dtype = np.int64, delimiter = '\t')
    
    constraint_constant_vector = np.stack(arrays = [constraint_constant_vector_numerator, constraint_constant_vector_denominator], axis = 0)
    
    print("read_constraint_constant_vector: constraint_constant_vector = \n{}".format(constraint_constant_vector[0].astype(np.float64) / constraint_constant_vector[1]))
    
    return constraint_constant_vector

### Get constraint coefficient matrix

In [62]:
# This function reads in the constraint coefficient matrix
def read_constraint_coefficient_matrix():
    # shape = (2, number_of_constraints, number_of_variables)
    constraint_coefficient_matrix_numerator = np.loadtxt(fname = "inputs/constraint_coefficient_matrix_numerator.txt", dtype = np.int64, delimiter = '\t')
    constraint_coefficient_matrix_denominator = np.loadtxt(fname = "inputs/constraint_coefficient_matrix_denominator.txt", dtype = np.int64, delimiter = '\t')
    
    constraint_coefficient_matrix = np.stack(arrays = [constraint_coefficient_matrix_numerator, constraint_coefficient_matrix_denominator], axis = 0)
    
    print("read_constraint_coefficient_matrix: constraint_coefficient_matrix = \n{}".format(constraint_coefficient_matrix[0].astype(np.float64) / constraint_coefficient_matrix[1]))
    
    return constraint_coefficient_matrix

# Optimal values

### Define placeholders for the optimal parameters

In [63]:
# This function creates the variables to hold the optimal opjective function value and associated variable values
def create_optimal_objective_function_and_variable_values(maximization_problem, number_of_variables):
    # shape = (2,)
    optimal_objective_function_value = np.ones(shape = [2], dtype = np.int64)
    optimal_objective_function_value[0] = np.where(maximization_problem == 1, np.iinfo(np.int64).min, np.iinfo(np.int64).max)

    # shape = (2, number_of_variables)
    optimal_variable_values = np.stack(arrays = [np.zeros(shape = [number_of_variables], dtype = np.int64), np.ones(shape = [number_of_variables], dtype = np.int64)], axis = 0)
    
    return optimal_objective_function_value, optimal_variable_values

# Mixed Integer Linear Programming

In [64]:
# This function performs mixed integer linear programming
def mixed_integer_linear_programming(maximization_problem, number_of_constraints, number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, objective_function_initial_constant, objective_function_coefficient_vector, constraint_inequality_direction_vector, constraint_constant_vector, constraint_coefficient_matrix, optimal_objective_function_value, optimal_variable_values):
    error_code = 0

    print("\n\n************************************************************************************************************\n")
    print("************************************************************************************************************\n")
    print("*************************************************** MILP ***************************************************\n")
    print("************************************************************************************************************\n")
    print("************************************************************************************************************\n\n\n")

    # modify_constraint_inequality_directions_and_count_directions
    number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints, modified_constraint_inequality_direction_vector = \
        modify_constraint_inequality_directions_and_count_directions(number_of_constraints, number_of_variables_required_to_be_binary, constraint_constant_vector, constraint_inequality_direction_vector)

    number_of_slack_surplus_variables = number_of_less_than_or_equal_to_constraints + number_of_greater_than_or_equal_to_constraints
    number_of_artificial_variables = number_of_equal_to_constraints + number_of_greater_than_or_equal_to_constraints

    # initialize_current_tableau_row_and_column_counts
    tableau_current_size = initialize_current_tableau_row_and_column_counts(number_of_constraints, number_of_variables, number_of_variables_required_to_be_binary, number_of_slack_surplus_variables, number_of_artificial_variables)

    # create_tableau_matrix
    number_of_constraints, tableau_matrix = create_tableau_matrix(maximization_problem, number_of_constraints, number_of_variables, number_of_variables_required_to_be_binary, number_of_slack_surplus_variables, number_of_artificial_variables, variable_special_requirements, objective_function_initial_constant, objective_function_coefficient_vector, modified_constraint_inequality_direction_vector, constraint_constant_vector, constraint_coefficient_matrix)

#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

    # create_basic_variables
    basic_variables = create_basic_variables(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix)

    # create_basic_feasible_solution
    basic_feasible_solution = create_basic_feasible_solution(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables)

    # write_initial_LP_values
    write_initial_LP_values(number_of_constraints, tableau_matrix, basic_variables, basic_feasible_solution)

    # print_initial_counts
    print_initial_counts(number_of_constraints, number_of_variables, number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_current_size)

#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

    # Solve LP Relaxation

    # This function finds the optimal solution for the given variables and constraints
    # simplex_algorithm
    error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size = simplex_algorithm(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size)

    # write_final_LP_values
    write_final_LP_values(number_of_constraints, tableau_matrix, basic_variables, basic_feasible_solution)

    if error_code == 0: # if LP is optimal then MILP will be either infeasible or optimal
        print("mixed_integer_linear_programming: LP is optimal!")

        # update_optimal_objective_function_and_variable_solution
        optimal_objective_function_value, optimal_variable_values = update_optimal_objective_function_and_variable_solution(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix)

        # print_optimal_results
        print_optimal_results(optimal_objective_function_value, optimal_variable_values)

        if number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary > 0: # if there is at least one variable required to be integer or binary
            print("\n\n************************************************************************************************************\n")
            print("************************************************************************************************************\n")
            print("*********************************************  BRANCH AND BOUND ********************************************\n")
            print("************************************************************************************************************\n")
            print("************************************************************************************************************\n\n\n")

            # branch_and_bound_MILP
            error_code, optimal_objective_function_value, optimal_variable_values = branch_and_bound_MILP(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code)

            if error_code == 0:
                print("mixed_integer_linear_programming: LP was optimal, MILP is optimal!")

                write_final_MILP_values(optimal_objective_function_value, optimal_variable_values)
            elif error_code == 1:
                print("mixed_integer_linear_programming: LP was optimal, MILP is infeasible!")
            elif error_code == 2:
                print("mixed_integer_linear_programming: LP was optimal, MILP is unbounded!  This is IMPOSSIBLE since LP was optimal!")
            else:
                print("mixed_integer_linear_programming: WEIRD ERROR_CODE = {}".format(error_code))
    elif error_code == 1: # if LP is infeasible then MILP will be infeasible since it is a subset of a NULL set
        print("mixed_integer_linear_programming: LP is infeasible!")

        if number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary > 0: # if there is at least one variable required to be integer or binary
            print("mixed_integer_linear_programming: An infeasible LP ALWAYS leads to an infeasible MILP!")
    elif error_code == 2: # if LP is unbounded and all LP coefficients are rational then the MILP will either be infeasible or unbounded, however if LP has some irrational coefficients then MILP might be optimal instead
        print("mixed_integer_linear_programming: LP is unbounded!")

        if number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary > 0: # if there is at least one variable required to be integer or binary
            print("\n\n************************************************************************************************************\n")
            print("************************************************************************************************************\n")
            print("*********************************************  BRANCH AND BOUND ********************************************\n")
            print("************************************************************************************************************\n")
            print("************************************************************************************************************\n\n\n")

            error_code, optimal_objective_function_value, optimal_variable_values = branch_and_bound_MILP(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, error_code)

            if error_code == 0:
                print("mixed_integer_linear_programming: LP was unbounded, MILP is optimal!  There must be some irrational coefficients and/or binary variables!")
            elif error_code == 1:
                print("mixed_integer_linear_programming: LP was unbounded, MILP is infeasible!")
            elif error_code == 2:
                print("mixed_integer_linear_programming: LP was unbounded, MILP is unbounded!")
            else:
                print("mixed_integer_linear_programming: WEIRD ERROR_CODE = {}".format(error_code))
    else:
        print("mixed_integer_linear_programming: WEIRD ERROR_CODE = {}".format(error_code))

#//////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

    return error_code, optimal_objective_function_value, optimal_variable_values

In [65]:
# This function modifies the constraint inequality directions and then counts instances of each direction type
def modify_constraint_inequality_directions_and_count_directions(number_of_constraints, number_of_variables_required_to_be_binary, constraint_constant_vector, constraint_inequality_direction_vector):
    number_of_less_than_or_equal_to_constraints = number_of_variables_required_to_be_binary
    number_of_equal_to_constraints = 0
    number_of_greater_than_or_equal_to_constraints = 0
    
    modified_constraint_inequality_direction_vector = np.concatenate([np.copy(a = constraint_inequality_direction_vector), np.ones(shape = [number_of_variables_required_to_be_binary], dtype = np.int64)], axis = 0)
    
    greater_than_or_equal_to_mask = constraint_inequality_direction_vector[0:number_of_constraints] == -1
    equal_to_mask = constraint_inequality_direction_vector[0:number_of_constraints] == 0
    less_than_or_equal_to_mask = constraint_inequality_direction_vector[0:number_of_constraints] == 1
    
    negative_constraint_constant_mask = constraint_constant_vector[0, 0:number_of_constraints] < 0
    
    # if originally greater than or equal to
    condition = np.all(a = [greater_than_or_equal_to_mask, negative_constraint_constant_mask], axis = 0)
    number_of_less_than_or_equal_to_constraints += np.count_nonzero(a = condition)
    
    modified_constraint_inequality_direction_vector[0:number_of_constraints] = np.where(condition, np.ones(shape = [number_of_constraints], dtype = np.int64), constraint_inequality_direction_vector[0:number_of_constraints])
    
    condition = np.all(a = [greater_than_or_equal_to_mask, np.invert(negative_constraint_constant_mask)], axis = 0)
    number_of_greater_than_or_equal_to_constraints += np.count_nonzero(a = condition)
    
    # if originally equal to
    condition = equal_to_mask
    number_of_equal_to_constraints = np.count_nonzero(a = condition)
    
    # if originally less than or equal to
    condition = np.all(a = [less_than_or_equal_to_mask, negative_constraint_constant_mask], axis = 0)
    number_of_greater_than_or_equal_to_constraints += np.count_nonzero(a = condition)
    
    modified_constraint_inequality_direction_vector[0:number_of_constraints] = np.where(condition, -np.ones(shape = [number_of_constraints], dtype = np.int64), constraint_inequality_direction_vector[0:number_of_constraints])
    
    condition = np.all(a = [less_than_or_equal_to_mask, np.invert(negative_constraint_constant_mask)], axis = 0)
    number_of_less_than_or_equal_to_constraints += np.count_nonzero(a = condition)
    
    return number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints, modified_constraint_inequality_direction_vector

In [66]:
# This function initializes the current tableau row and column counts
def initialize_current_tableau_row_and_column_counts(number_of_constraints, number_of_variables, number_of_variables_required_to_be_binary, number_of_slack_surplus_variables, number_of_artificial_variables):
    tableau_current_rows = np.where(number_of_artificial_variables > 0, number_of_constraints + number_of_variables_required_to_be_binary + 2, number_of_constraints + number_of_variables_required_to_be_binary + 1)
    tableau_current_columns = number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables + 1
    
    tableau_current_size = np.stack(arrays = [tableau_current_rows, tableau_current_columns], axis = 0)
    
    return tableau_current_size

In [67]:
# This function creates the simplex tableau matrix
def create_tableau_matrix(maximization_problem, number_of_constraints, number_of_variables, number_of_variables_required_to_be_binary, number_of_slack_surplus_variables, number_of_artificial_variables, variable_special_requirements, objective_function_initial_constant, objective_function_coefficient_vector, constraint_inequality_direction_vector, constraint_constant_vector, constraint_coefficient_matrix):
    """
        Tableau generic format is:

        con\var   |   b    x1    x2    ...   xn    s1       s2       ...   sq       A1    A2    ...      Ar
        ------------------------------------------------------------------------------------------------------------------------
        con1      |   b1   x11   x11   ...   x1n   0/1/-1   0/1/-1   ...   0/1/-1   0/1   0/1   ...      0/1
        con2      |   b2   x21   x22   ...   x2n   0/1/-1   0/1/-1   ...   0/1/-1   0/1   0/1   ...      0/1
        .         |   .    .     .     ...   .     .        .        ...   .        .     .     ...      .
        .         |   .    .     .     ...   .     .        .        ...   .        .     .     ...      .
        .         |   .    .     .     ...   .     .        .        ...   .        .     .     ...      .
        conm      |   bm   xm1   xm2   ...   xmn   0/1/-1   0/1/-1   ...   0/1/-1   0/1   0/1   ...      0/1
        z         |   z0   -c1   -c2   ...   0     0        0        ...   0        0     0     ...      0
        zA        |   0    0     0     ...   0     0        0        ...   0        -1    -1    ...      -1
    """

    # First add b constant vector
    b_constant_vector = np.stack(arrays = [np.expand_dims(a = np.where(constraint_constant_vector[0, :] < 0, -constraint_constant_vector[0, :], constraint_constant_vector[0, :]), axis = -1), np.expand_dims(a = constraint_constant_vector[1, :], axis = -1)], axis = 0)
    
    # Next add b constant vector for binary required variables
    b_constant_vector_binary = np.ones(shape = [2, number_of_variables_required_to_be_binary, 1], dtype = np.int64)

    # Next add A matrix
    condition = constraint_constant_vector[0, :] < 0
    true_array = -constraint_coefficient_matrix[0, :, :]
    false_array = constraint_coefficient_matrix[0, :, :]
    a_matrix = np.stack(arrays = [np.stack(arrays = list(map(lambda i: np.where(condition[i], true_array[i, :], false_array[i, :]), np.arange(number_of_constraints))), axis = 0), constraint_coefficient_matrix[1, :, :]], axis = 0)

    # Next add A matrix for binary required variables
    a_matrix_binary_numerator = np.array(object = list(map(lambda i: np.identity(n = number_of_variables)[i, :], np.where(variable_special_requirements[:] == 2)[0])))
    a_matrix_binary_denominator = np.array(object = list(map(lambda i: np.ones(shape = [number_of_variables, number_of_variables])[i, :], np.where(variable_special_requirements[:] == 2)[0])))
    a_matrix_binary = np.array(object = [a_matrix_binary_numerator, a_matrix_binary_denominator], dtype = np.int64).reshape(2, number_of_variables_required_to_be_binary, number_of_variables)
            
    # Next add slack/surplus variable matrix
    condition = constraint_inequality_direction_vector[:] == 1
    true_array = np.identity(n = number_of_constraints + number_of_variables_required_to_be_binary, dtype = np.int64)
    false_array = -np.identity(n = number_of_constraints + number_of_variables_required_to_be_binary, dtype = np.int64)
    slack_surplus_matrix = np.stack(arrays = [np.where(condition, true_array, false_array), np.ones(shape = [number_of_constraints + number_of_variables_required_to_be_binary, number_of_constraints + number_of_variables_required_to_be_binary], dtype = np.int64)], axis = 0)
            
    # Next add artificial variable matrix
    condition = constraint_inequality_direction_vector[:] != 1
    true_array = np.identity(n = number_of_constraints + number_of_variables_required_to_be_binary, dtype = np.int64)
    false_array = np.zeros(shape = [number_of_constraints + number_of_variables_required_to_be_binary, number_of_constraints + number_of_variables_required_to_be_binary], dtype = np.int64)
    temp_artificial_variable_matrix = np.stack(arrays = [np.where(condition, true_array, false_array), np.ones(shape = [number_of_constraints + number_of_variables_required_to_be_binary, number_of_constraints + number_of_variables_required_to_be_binary], dtype = np.int64)], axis = 0)
    artificial_variable_matrix = temp_artificial_variable_matrix[:, :, condition]
    
    # Next add original -cT vector
    original_ct_vector = np.expand_dims(a = np.stack(arrays = [np.where(maximization_problem == 1, -objective_function_coefficient_vector[0, :], objective_function_coefficient_vector[0, :]), np.ones(shape = [number_of_variables], dtype = np.int64)], axis = 0), axis = 1)
    
    # Next add artificial -cT vector
    artificial_ct_vector = np.expand_dims(a = np.stack(arrays = [-np.ones(shape = [number_of_artificial_variables], dtype = np.int64), np.ones(shape = [number_of_artificial_variables], dtype = np.int64)], axis = 0), axis = 1)

    # Lastly add initial z
    initial_z = np.copy(a = objective_function_initial_constant).reshape(2, 1, 1)
    
    # Now construct tableau with all of the pieces
    b_constant_column = np.concatenate([b_constant_vector, b_constant_vector_binary], axis = 1)
    
    a_matrix_full = np.concatenate([a_matrix, a_matrix_binary], axis = 1)
    
    non_artificial_tableau_above_objective_rows = np.concatenate([b_constant_column, a_matrix_full, slack_surplus_matrix], axis = 2)
    
    artificial_tableau_above_objective_rows = artificial_variable_matrix
    
    objective_row = np.concatenate([initial_z, original_ct_vector, np.stack(arrays = [np.zeros(shape = [1, number_of_slack_surplus_variables], dtype = np.int64), np.ones(shape = [1, number_of_slack_surplus_variables], dtype = np.int64)], axis = 0)], axis = 2)
    
    non_artificial_tableau = np.concatenate([non_artificial_tableau_above_objective_rows, objective_row], axis = 1)
    
    artificial_tableau = np.concatenate([artificial_tableau_above_objective_rows, np.array(object = [np.zeros(shape = [1, number_of_artificial_variables], dtype = np.int64), np.ones(shape = [1, number_of_artificial_variables], dtype = np.int64)], dtype = np.int64)], axis = 1)
    
    combined_tableau = np.concatenate([non_artificial_tableau, artificial_tableau], axis = 2)
    
    if number_of_artificial_variables > 0:
        artificial_objective_row = np.concatenate([np.stack(arrays = [np.zeros(shape = [1, 1 + number_of_variables + number_of_slack_surplus_variables], dtype = np.int64), np.ones(shape = [1, 1 + number_of_variables + number_of_slack_surplus_variables], dtype = np.int64)], axis = 0), artificial_ct_vector], axis = 2)
    
        tableau_matrix = np.concatenate([combined_tableau, artificial_objective_row], axis = 1)
    else:
        tableau_matrix = combined_tableau
    
    number_of_constraints += number_of_variables_required_to_be_binary
    
    return number_of_constraints, tableau_matrix

In [68]:
# This function creates the basic variables which tell which basis we currently are in
def create_basic_variables(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix):
    basic_variables = np.zeros(shape = [number_of_constraints], dtype = np.int64)
    
    condition = tableau_matrix[0, :number_of_constraints, 1:1 + number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables] > 0
    
    number_of_positive_elements = np.count_nonzero(a = condition, axis = 0)
    
    positive_element_column_indices = np.extract(condition = number_of_positive_elements == 1, arr = np.arange(number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables))
    
    positive_element_column_values = condition[:, positive_element_column_indices]
    
    positive_element_row_indices = np.squeeze(a = np.stack(list(map(lambda i: np.where(positive_element_column_values[:, i])[0], np.arange(number_of_constraints))), axis = 0), axis = 1)
    
    basic_variables[positive_element_row_indices] = positive_element_column_indices + 1
            
    return basic_variables

In [69]:
# This function creates the basic feasible solution to keep track of the best variable values in the current basis
def create_basic_feasible_solution(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables):
    basic_feasible_solution_numerator = np.zeros(shape = [number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables], dtype = np.int64)
    basic_feasible_solution_denominator = np.ones(shape = [number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables], dtype = np.int64)
    basic_feasible_solution = np.stack(arrays = [basic_feasible_solution_numerator, basic_feasible_solution_denominator], axis = 0)

    basic_feasible_solution_basic_variable_values = np.stack(arrays = list(map(lambda i, j: rational_division(tableau_matrix[0, i, 0], tableau_matrix[1, i, 0], tableau_matrix[0, i, j], tableau_matrix[1, i, j]), np.arange(number_of_constraints), basic_variables[:number_of_constraints])), axis = 1)
        
    basic_feasible_solution[:, basic_variables[:number_of_constraints] - 1] = basic_feasible_solution_basic_variable_values

    return basic_feasible_solution

In [70]:
# This function writes to disk the initial LP array values
def write_initial_LP_values(number_of_constraints, tableau_matrix, basic_variables, basic_feasible_solution):
    print("write_initial_LP_values: objective function optimal value = \n{}".format(tableau_matrix[0, number_of_constraints, 0].astype(np.float64) / tableau_matrix[1, number_of_constraints, 0]))
    print("write_initial_LP_values: basic variables = \n{}".format(basic_variables))
    print("write_initial_LP_values: basic feasible solution = \n{}".format(basic_feasible_solution[0, :].astype(np.float64) / basic_feasible_solution[1, :]))
    print("write_initial_LP_values: tableau matrix = \n{}".format(tableau_matrix[0, :, :].astype(np.float64) / tableau_matrix[1, :, :]))
    
    return

In [71]:
# This function prints the initial constraint, variable, etc. counts
def print_initial_counts(number_of_constraints, number_of_variables, number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_current_size):
    print("print_initial_counts: number_of_constraints = {} & number_of_variables = {}".format(number_of_constraints, number_of_variables))
    print("print_initial_counts: number_of_less_than_or_equal_to_constraints = {}, number_of_equal_to_constraints = {}, number_of_greater_than_or_equal_to_constraints = {}".format(number_of_less_than_or_equal_to_constraints, number_of_equal_to_constraints, number_of_greater_than_or_equal_to_constraints))
    print("print_initial_counts: number_of_slack_surplus_variables = {} & number_of_artificial_variables = {}".format(number_of_slack_surplus_variables, number_of_artificial_variables))
    print("print_initial_counts: tableau_current_size = {}".format(tableau_current_size))
    
    return

In [72]:
# This function writes to disk the final LP array values
def write_final_LP_values(number_of_constraints, tableau_matrix, basic_variables, basic_feasible_solution):
    print("write_final_LP_values: objective function optimal value = \n{}".format(tableau_matrix[0, number_of_constraints, 0].astype(np.float64) / tableau_matrix[1, number_of_constraints, 0]))
    print("write_final_LP_values: basic variables = \n{}".format(basic_variables))
    print("write_final_LP_values: basic feasible solution = \n{}".format(basic_feasible_solution[0, :].astype(np.float64) / basic_feasible_solution[1, :]))
    print("write_final_LP_values: tableau matrix = \n{}".format(tableau_matrix[0, :, :].astype(np.float64) / tableau_matrix[1, :, :]))
    
    return

In [73]:
# This function writes to disk the final MILP array values
def write_final_MILP_values(optimal_objective_function_value, optimal_variable_values):
    print("write_final_MILP_values: optimal_objective_function_value = \n{}".format(optimal_objective_function_value[0].astype(np.float64) / optimal_objective_function_value[1]))
    print("write_final_MILP_values: optimal_variable_values = \n{}".format(optimal_variable_values[0, :].astype(np.float64) / optimal_variable_values[1, :]))
    
    return

In [74]:
# This function updates the optimal objective function and variable solution
def update_optimal_objective_function_and_variable_solution(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix):
    optimal_objective_function_value = np.where(maximization_problem == 1, tableau_matrix[:, number_of_constraints, 0], np.stack(arrays = [-tableau_matrix[0, number_of_constraints, 0], tableau_matrix[1, number_of_constraints, 0]], axis = 0))

    optimal_variable_values = basic_feasible_solution[:, :number_of_variables]

    return optimal_objective_function_value, optimal_variable_values

# Simplex

In [75]:
# This function finds the optimal solution for the given variables and constraints
def simplex_algorithm(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size):
    error_code = 0

    #*********************************************************************************
    #*********************************** PHASE 1 *************************************
    #*********************************************************************************

    if number_of_artificial_variables > 0:
        # This function transforms the tableau by removing artificial variables to obtain a basic feasible solution
        error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size = simplex_phase_1(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size)
        tableau_current_size[0] -= 1 # decrement current rows since we've eliminated artificial objective function row
        tableau_matrix = tableau_matrix[:, 0:tableau_current_size[0], 0:tableau_current_size[1]]

    #*********************************************************************************
    #*********************************** PHASE 2 *************************************
    #*********************************************************************************

    if error_code == 0:
        # This function starts from an basic feasible solution and iterates toward the optimal solution
        error_code, basic_variables, basic_feasible_solution, tableau_matrix = simplex_phase_2(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size)

    #*********************************************************************************
    #*********************************** SOLUTION ************************************
    #*********************************************************************************

    if error_code == 0:
        basic_feasible_solution = update_basic_feasible_solution(number_of_variables + number_of_slack_surplus_variables, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix)

    return error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size

In [76]:
# This function transforms the tableau by removing artificial variables to obtain a basic feasible solution
def simplex_phase_1(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size):
    error_code = 0

    # PHASE I (first find the feasible region)
    
    # Remove artificial variables from objective function
    artificial_variable_col_index = np.arange(start = number_of_variables + number_of_slack_surplus_variables + 1, stop = number_of_variables + number_of_slack_surplus_variables + 1 + number_of_artificial_variables)

    artificial_variable_row_numerator_mask = np.squeeze(a = tableau_matrix[0, 0:tableau_current_size[0] - 2, artificial_variable_col_index] == 1)
    artificial_variable_row_denominator_mask = np.squeeze(a = tableau_matrix[1, 0:tableau_current_size[0] - 2, artificial_variable_col_index]) == 1
    
    condition = np.all(a = [artificial_variable_row_numerator_mask, artificial_variable_row_denominator_mask], axis = 0)

    artificial_variable_row_index = np.extract(condition = condition, arr = np.tile(A = np.arange(tableau_current_size[0] - 2), reps = (number_of_artificial_variables, 1)))
    
    updated_artificial_objective_row = tableau_matrix[:, number_of_constraints + 1, :]
    
    i = 0
    while i < number_of_artificial_variables:
        updated_artificial_objective_row = np.stack(arrays = list(map(lambda j: rational_addition(updated_artificial_objective_row[0, j], updated_artificial_objective_row[1, j], tableau_matrix[0, artificial_variable_row_index[i], j], tableau_matrix[1, artificial_variable_row_index[i], j]), np.arange(tableau_current_size[1]))), axis = 1)
        i += 1
    
    tableau_matrix[:, number_of_constraints + 1, 0:tableau_current_size[1]] = updated_artificial_objective_row
        
    # Update Basic Feasible Solution
    basic_feasible_solution = update_basic_feasible_solution(tableau_current_size[1] - 1, tableau_current_size[0] - 2, basic_variables, basic_feasible_solution, tableau_matrix)

    # Count initial number of positive elements in the objective function row
    number_of_positive_objective_function_elements = np.count_nonzero(a = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]] > 0)

    iteration = 0
    while number_of_positive_objective_function_elements > 0 and number_of_artificial_variables > 0 and error_code == 0:
        # Find most positive objective function element amongst the variables which will become our entering variable
        objective_function_row = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]].astype(np.float64) / tableau_matrix[1, tableau_current_size[0] - 1, 1:tableau_current_size[1]]
        most_positive_objective_function_element_value = np.max(a = objective_function_row)
        most_positive_objective_function_element_index = np.argmax(a = objective_function_row) + 1

        if most_positive_objective_function_element_value > 0: # if a pivot column was found
            pivot_col_index = most_positive_objective_function_element_index

            # Search for smallest non negative ratio of bi / aij which will be departing variable
            condition = tableau_matrix[0, 0:tableau_current_size[0] - 2, pivot_col_index] > 0 # pivot element needs to be positive
            b_a_indices = np.arange(tableau_current_size[0] - 2)[condition]
            b_a_ratio = np.stack(arrays = list(map(lambda i: rational_division(tableau_matrix[0, i, 0], tableau_matrix[1, i, 0], tableau_matrix[0, i, pivot_col_index], tableau_matrix[1, i, pivot_col_index]), b_a_indices)), axis = 1)
            b_a_ratio_double = b_a_ratio[0, :].astype(np.float64) / b_a_ratio[1, :]
            
            condition = b_a_ratio_double >= 0
            non_negative_b_a_ratio_indices = np.arange(b_a_ratio_double.shape[0])[condition]
            non_negative_b_a_ratio_values_double = b_a_ratio_double[condition]
            non_negative_b_a_ratio_values = b_a_ratio[:, condition]

            smallest_non_negative_ratio_value_double = np.min(a = non_negative_b_a_ratio_values_double)
            smallest_non_negative_ratio_value_index = np.argmin(a = non_negative_b_a_ratio_values_double)
            smallest_non_negative_ratio_value = non_negative_b_a_ratio_values[:, smallest_non_negative_ratio_value_index]
            smallest_non_negative_ratio_index = b_a_indices[non_negative_b_a_ratio_indices[smallest_non_negative_ratio_value_index]] # this should already be using Bland's Rule since it is taking the lowest index in ties to avoid cycles
            
            condition = basic_variables[b_a_indices] >= number_of_variables + number_of_slack_surplus_variables
            non_negative_b_a_ratio_artificial_indices = np.arange(b_a_ratio_double.shape[0])[condition]
            non_negative_b_a_ratio_artificial_values_double = b_a_ratio_double[condition]
            non_negative_b_a_ratio_artificial_values = b_a_ratio[:, condition]
            
            smallest_artificial_variable_non_negative_ratio_value_double = np.min(a = non_negative_b_a_ratio_artificial_values_double)
            smallest_artificial_variable_non_negative_ratio_value_index = np.argmin(a = non_negative_b_a_ratio_artificial_values_double)
            smallest_artificial_variable_non_negative_ratio_value = non_negative_b_a_ratio_artificial_values[:, smallest_artificial_variable_non_negative_ratio_value_index]
            smallest_artificial_variable_non_negative_ratio_index = b_a_indices[non_negative_b_a_ratio_artificial_indices[smallest_artificial_variable_non_negative_ratio_value_index]] # this should already be using Bland's Rule since it is taking the lowest index in ties to avoid cycles
            
            if smallest_non_negative_ratio_value.size != 0: # if pivot row was found
                # IF LOWEST RATIO OCCURS BOTH IN AN ARTIFICIAL VARIABLE ROW AND A NON-NEGATIVE VARIABLE ROW THEN MUST CHOOSE PIVOT ROW AS NEGATIVE VARIABLE ROW
                if smallest_artificial_variable_non_negative_ratio_index == 0: # if there are NO more artificial variables
                    pivot_row_index = smallest_non_negative_ratio_index
                else: # if there are still artificial variables
                    if smallest_non_negative_ratio_index != smallest_artificial_variable_non_negative_ratio_index:
                        if np.all(a = [smallest_non_negative_ratio_value[0] == smallest_artificial_variable_non_negative_ratio_value[0], smallest_non_negative_ratio_value[1] == smallest_artificial_variable_non_negative_ratio_value[1]]):
                            pivot_row_index = smallest_artificial_variable_non_negative_ratio_index
                        else:
                            pivot_row_index = smallest_non_negative_ratio_index
                    else:
                        pivot_row_index = smallest_non_negative_ratio_index

                pivot_value = tableau_matrix[:, pivot_row_index, pivot_col_index]

                condition = basic_variables[pivot_row_index] >= number_of_variables + number_of_slack_surplus_variables
                if condition: # if basic variable is an artificial variable
                    number_of_artificial_variables -= 1
                    tableau_current_size[1] -= 1

                # Remove departing variable from and add entering variable to basic variables
                basic_variables[pivot_row_index] = pivot_col_index

                # This function performs Gauss-Jordan Elimination on the pivot column
                tableau_matrix = pivot_column_gauss_jordan_elimnation(tableau_current_size, pivot_row_index, pivot_col_index, pivot_value, tableau_matrix)

                # This function updates the basic feasible solution
                basic_feasible_solution = update_basic_feasible_solution(tableau_current_size[1] - 1, tableau_current_size[0] - 2, basic_variables, basic_feasible_solution, tableau_matrix)

                # Count again the number of variables that are positive still
                condition = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]] > 0
                number_of_positive_objective_function_elements = np.count_nonzero(a = condition)

                iteration += 1
            else:
                error_code = 1 # infeasible
        else:
            error_code = 1 # infeasible

    if number_of_artificial_variables > 0:
        print("simplex_phase_1: ERROR: No basic feasible solution! number_of_artificial_variables = {}".format(number_of_artificial_variables))
        error_code = 1

    return error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size

In [77]:
# This function starts from an basic feasible solution and iterates toward the optimal solution
def simplex_phase_2(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size):
    error_code = 0
    old_optimum = np.finfo(np.float64).min

    # PHASE II (find the optimal solution in the feasible region we found above)

    # Count initial number of negative elements in the objective function row
    number_of_negative_objective_function_elements = np.count_nonzero(a = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]] < 0)

    while number_of_negative_objective_function_elements > 0 and error_code == 0: # while there is still a optimal solution
        # Search for most negative element in objective function row
        objective_function_row = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]].astype(np.float64) / tableau_matrix[1, tableau_current_size[0] - 1, 1:tableau_current_size[1]]
        most_negative_objective_function_element_value = np.min(a = objective_function_row)
        most_negative_objective_function_element_index = np.argmin(a = objective_function_row) + 1

        if most_negative_objective_function_element_value < 0: # if pivot column was found
            pivot_col_index = most_negative_objective_function_element_index

            # Check to see if the problem is unbounded
            number_of_positive_elements = np.count_nonzero(a = tableau_matrix[0, 0:tableau_current_size[0] - 1, pivot_col_index] > 0)

            if number_of_positive_elements > 0: # if the problem is bounded still
                # Search for smallest non negative ratio of bi / aij
                condition = tableau_matrix[0, 0:tableau_current_size[0] - 1, pivot_col_index] > 0 # pivot element needs to be positive
                b_a_indices = np.extract(condition = condition, arr = np.arange(tableau_current_size[0] - 1))
                b_a_numerator = np.extract(condition = condition, arr = tableau_matrix[0, 0:tableau_current_size[0] - 1, 0].astype(np.float64) / tableau_matrix[1, 0:tableau_current_size[0] - 1, 0])
                b_a_denominator = np.extract(condition = condition, arr = tableau_matrix[0, 0:tableau_current_size[0] - 1, pivot_col_index].astype(np.float64) / tableau_matrix[1, 0:tableau_current_size[0] - 1, pivot_col_index])
                b_a_ratio = b_a_numerator / b_a_denominator

                non_negative_b_a_ratio_indices = np.extract(condition = b_a_ratio >= 0, arr = np.arange(b_a_ratio.shape[0]))
                non_negative_b_a_ratio_values = np.extract(condition = b_a_ratio >= 0, arr = b_a_ratio)

                smallest_non_negative_ratio_value = np.min(a = non_negative_b_a_ratio_values)
                smallest_non_negative_ratio_index = b_a_indices[non_negative_b_a_ratio_indices[np.argmin(a = non_negative_b_a_ratio_values)]] # this should already be using Bland's Rule since it is taking the lowest index in ties to avoid cycles
                
                if smallest_non_negative_ratio_value.size != 0: # if pivot row was found
                    pivot_row_index = smallest_non_negative_ratio_index

                    pivot_value = tableau_matrix[:, pivot_row_index, pivot_col_index]

                    # Remove departing variable from and add entering variable to basic variables
                    basic_variables[pivot_row_index] = pivot_col_index

                    # This function performs Gauss-Jordan Elimination on the pivot column
                    tableau_matrix = pivot_column_gauss_jordan_elimnation(tableau_current_size, pivot_row_index, pivot_col_index, pivot_value, tableau_matrix)
                    
                    # This function updates the basic feasible solution
                    basic_feasible_solution = update_basic_feasible_solution(tableau_current_size[1] - 1, tableau_current_size[0] - 1, basic_variables, basic_feasible_solution, tableau_matrix)
                    
                    # Count new number of negative elements in the objective function row
                    number_of_negative_objective_function_elements = np.count_nonzero(a = tableau_matrix[0, tableau_current_size[0] - 1, 1:tableau_current_size[1]] < 0)
                    
                    if old_optimum == tableau_matrix[0, tableau_current_size[0] - 1, 0].astype(np.float64) / tableau_matrix[1, tableau_current_size[0] - 1, 0]:
                        print("simplex_phase_2: NOTE: Problem is Degenerate!\n")
                    else:
                        old_optimum = tableau_matrix[0, tableau_current_size[0] - 1, 0].astype(np.float64) / tableau_matrix[1, tableau_current_size[0] - 1, 0]
                else:
                    error_code = 1 # infeasible
            else:
                print("simplex_phase_2: ERROR: Problem is Unbounded!\n")
                error_code = 2 # unbounded
                break # break while loop
        else:
            error_code = 1 # infeasible

    return error_code, basic_variables, basic_feasible_solution, tableau_matrix

In [78]:
# This function performs Gauss-Jordan Elimination on the pivot column
def pivot_column_gauss_jordan_elimnation(matrix_size, pivot_row_index, pivot_col_index, pivot_value, tableau_matrix):
    # Perform Gauss-Jordan Elimination on pivot column around pivot row
    number_of_rows = matrix_size[0]
    number_of_columns = matrix_size[1]
    
    # First take care of pivot row
    condition = np.any(a = tableau_matrix[:, pivot_row_index, pivot_col_index] != 1)
    
    true_array = np.stack(arrays = list(map(lambda i: rational_division(tableau_matrix[0, pivot_row_index, i], tableau_matrix[1, pivot_row_index, i], pivot_value[0], pivot_value[1]), np.arange(number_of_columns))), axis = 1)
    
    false_array = tableau_matrix[:, pivot_row_index, :number_of_columns]
    
    tableau_matrix_pivot_row_fixed = np.where(condition, true_array, false_array)
        
    tableau_matrix[:, pivot_row_index, :number_of_columns] = tableau_matrix_pivot_row_fixed

    # Now take care of other rows
    cond1 = np.arange(number_of_rows) != pivot_row_index
    
    cond2 = tableau_matrix[0, :number_of_rows, pivot_col_index] != 0
    
    condition = np.all(a = [np.arange(number_of_rows) != pivot_row_index, tableau_matrix[0, :number_of_rows, pivot_col_index] != 0], axis = 0)
    
    true_array = tableau_matrix[:, :number_of_rows, pivot_col_index]
    
    false_array = np.stack(arrays = [np.zeros(shape = [number_of_rows], dtype = np.int64), np.ones(shape = [number_of_rows], dtype = np.int64)], axis = 0)
    
    pivot_value = np.where(condition, true_array, false_array)
    
    true_array = np.stack(arrays = list(map(lambda i: np.stack(arrays = list(map(lambda j: rational_addition(tableau_matrix[0, i, j], tableau_matrix[1, i, j], -pivot_value[0, i] * tableau_matrix[0, pivot_row_index, j], pivot_value[1, i] * tableau_matrix[1, pivot_row_index, j]), np.arange(number_of_columns))), axis = 1), np.arange(number_of_rows))), axis = 1)
    
    false_array = tableau_matrix[:, :, :number_of_columns]
    
    tableau_matrix_other_rows_fixed = np.stack(arrays = list(map(lambda i: np.where(condition[i], true_array[:, i, :], false_array[:, i, :]), np.arange(number_of_rows))), axis = 1)
    
    tableau_matrix[:, :number_of_rows, :number_of_columns] = tableau_matrix_other_rows_fixed
    
    return tableau_matrix

In [79]:
# This function updates the basic feasible solution
def update_basic_feasible_solution(number_of_total_variables, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix):
    basic_feasible_solution[0, :number_of_total_variables] = 0
    basic_feasible_solution[1, :number_of_total_variables] = 1

    basic_feasible_solution[:, basic_variables[:number_of_constraints] - 1] = np.stack(arrays = list(map(lambda i, j: rational_division(tableau_matrix[0, i, 0], tableau_matrix[1, i, 0], tableau_matrix[0, i, j], tableau_matrix[1, i, j]), np.arange(number_of_constraints), basic_variables[:number_of_constraints])), axis = 1)
    
    return basic_feasible_solution

# Branch and bound

In [80]:
# This function initiates the branch and bound
def branch_and_bound_MILP(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, optimal_objective_function_value, optimal_variable_values, objective_function_coefficient_vector, lp_error_code):
    number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary = 0
    last_variable_that_still_needs_to_become_integer_or_binary_index = 0

    error_code = 0

    # count_number_of_variables_needing_to_be_integer_or_binary_that_already_are
    number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index = \
        count_number_of_variables_needing_to_be_integer_or_binary_that_already_are(number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, basic_feasible_solution)

    if number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary < number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary:
        # Create variables to track the best optimal values and the corresponding variables
        best_milp_error_code = 1

        best_mixed_integer_optimal_value = np.array(object = [0, 1], dtype = np.int64)

        if maximization_problem == 1:
            best_mixed_integer_optimal_value_double = np.finfo(np.float64).min # negative infinity
            best_mixed_integer_optimal_value[0] = np.iinfo(np.int64).min
        else:
            best_mixed_integer_optimal_value_double = np.finfo(np.float64).max # positive infinity
            best_mixed_integer_optimal_value[0] = np.iinfo(np.int64).max
            
        best_mixed_integer_variable_values = np.stack(arrays = [np.zeros(shape = [number_of_variables], dtype = np.int64), np.ones(shape = [number_of_variables], dtype = np.int64)], axis = 0)

        # Call recursive branch and bound function
        # branch_and_bound_MILP_recursive
        error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
            branch_and_bound_MILP_recursive(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, 0)

        error_code = best_milp_error_code

        if best_milp_error_code == 0: # feasible
            print("\nbranch_and_bound_MILP: Optimal MILP!")
            # Update overall optimal solution
            optimal_objective_function_value = best_mixed_integer_optimal_value[:]
            optimal_variable_values[:, :number_of_variables] = best_mixed_integer_variable_values[:, :number_of_variables]
        elif best_milp_error_code == 1: # infeasible
            print("\nbranch_and_bound_MILP: Infeasible MILP!")
        elif best_milp_error_code == 2: # unbounded
            print("\nbranch_and_bound_MILP: Unbounded MILP!")
    else: # if all variables that need to be integer or binary are already
        if lp_error_code == 0:
            error_code = 0

            print("\nbranch_and_bound_MILP: If there were any integer or binary constraints, they were already satisfied before MILP BB by optimal LP!")
        elif lp_error_code == 2:
            error_code = 2

            print("\nbranch_and_bound_MILP: LP was unbounded and since computers can't represent irrational numbers, then MILP is unbounded!")
        else:
            error_code = 999
            print("\nbranch_and_bound_MILP: HOW DID I GET IN MILP BB?! ERROR!")

    return error_code, optimal_objective_function_value, optimal_variable_values

In [81]:
# This function counts the number of variables that need to be integer or binary AND already are
def count_number_of_variables_needing_to_be_integer_or_binary_that_already_are(number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, basic_feasible_solution):
    number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary = 0
    integer_last_variable_that_still_needs_to_become_integer_or_binary_index = -1
    binary_last_variable_that_still_needs_to_become_integer_or_binary_index = -1
    
    if number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary > 0:
        # Integer
        integer_variable_requirement_mask = variable_special_requirements[:] == 1

        basic_feasible_solution_denominator_equals_one_mask = basic_feasible_solution[1, :number_of_variables] == 1

        number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary += np.count_nonzero(a = np.all(a = [integer_variable_requirement_mask, basic_feasible_solution_denominator_equals_one_mask], axis = 0))

        integer_still_needs_all_array = np.all(a = [integer_variable_requirement_mask, np.invert(basic_feasible_solution_denominator_equals_one_mask)], axis = 0)
        
        integer_indices = np.where(integer_still_needs_all_array)[0]
        
        integer_last_variable_that_still_needs_to_become_integer_or_binary_index = np.max(np.concatenate([integer_indices, -np.ones(shape = [1], dtype = np.int64)], axis = 0))
        
        # Binary
        binary_variable_requirement_mask = variable_special_requirements[:] == 2

        basic_feasible_solution_numerator_equals_zero_mask = basic_feasible_solution[0, :number_of_variables] == 0

        basic_feasible_solution_numerator_equals_denominator_mask = basic_feasible_solution[0, :number_of_variables] == basic_feasible_solution[1, :number_of_variables]

        count_any_array = np.any(a = [basic_feasible_solution_numerator_equals_zero_mask, basic_feasible_solution_numerator_equals_denominator_mask], axis = 0)

        count_all_array = np.all(a = [binary_variable_requirement_mask, count_any_array], axis = 0)
        
        number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary += np.count_nonzero(a = count_all_array)

        basic_feasible_solution_float_between_zero_and_one_mask = np.all(a = [basic_feasible_solution[0, :number_of_variables].astype(np.float64) / basic_feasible_solution[1, :number_of_variables] >= 0, basic_feasible_solution[0, :number_of_variables].astype(np.float64) / basic_feasible_solution[1, :number_of_variables] <= 1], axis = 0)

        binary_still_needs_all_array = np.all(a = [binary_variable_requirement_mask, np.invert(basic_feasible_solution_numerator_equals_zero_mask), np.invert(basic_feasible_solution_numerator_equals_denominator_mask), basic_feasible_solution_float_between_zero_and_one_mask], axis = 0)

        binary_indices = np.where(binary_still_needs_all_array)[0]
        
        binary_last_variable_that_still_needs_to_become_integer_or_binary_index = np.max(np.concatenate([binary_indices, -np.ones(shape = [1], dtype = np.int64)], axis = 0))

        last_variable_that_still_needs_to_become_integer_or_binary_index = np.max([integer_last_variable_that_still_needs_to_become_integer_or_binary_index, binary_last_variable_that_still_needs_to_become_integer_or_binary_index])
        
    return number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index

In [82]:
# This function recursively applies branch and bound
def branch_and_bound_MILP_recursive(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, recursion_level):
    error_code = 0

    # Create old arrays so that we can reset back to how it was later
    tableau_current_size_old, number_of_constraints_old, number_of_slack_surplus_variables_old, number_of_artificial_variables_old = create_cache_values_for_branch_and_bound_MILP_recursion(tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables)
    
    basic_variables_old, basic_feasible_solution_old, tableau_matrix_old = create_cache_arrays_for_branch_and_bound_MILP_recursion(basic_variables, basic_feasible_solution, tableau_matrix)

    # This will be the value that we try next for the corresponding variable
    last_variable_that_still_needs_to_become_integer_value = basic_feasible_solution[0, last_variable_that_still_needs_to_become_integer_or_binary_index].astype(np.float64) / basic_feasible_solution[1, last_variable_that_still_needs_to_become_integer_or_binary_index]

    #*******************************************************************************
    #********************************** LESS THAN **********************************
    #*******************************************************************************

    # branch_and_bound_MILP_recursive_less_or_greater_than
    error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
        branch_and_bound_MILP_recursive_less_or_greater_than(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, last_variable_that_still_needs_to_become_integer_value, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, tableau_current_size_old, number_of_constraints_old, number_of_slack_surplus_variables_old, number_of_artificial_variables_old, basic_feasible_solution_old, basic_variables_old, tableau_matrix_old, 1, recursion_level)
    
    #*******************************************************************************
    #******************************** GREATER THAN *********************************
    #*******************************************************************************

    # branch_and_bound_MILP_recursive_less_or_greater_than
    error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
        branch_and_bound_MILP_recursive_less_or_greater_than(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, last_variable_that_still_needs_to_become_integer_value, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, tableau_current_size_old, number_of_constraints_old, number_of_slack_surplus_variables_old, number_of_artificial_variables_old, basic_feasible_solution_old, basic_variables_old, tableau_matrix_old, -1, recursion_level)
    
    return error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values

In [83]:
# This function creates cache values to reset the active values at the end of each recursion level
def create_cache_values_for_branch_and_bound_MILP_recursion(tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables):
    tableau_current_size_old = np.copy(a = tableau_current_size)
    number_of_constraints_old = np.copy(a = number_of_constraints)
    number_of_slack_surplus_variables_old = np.copy(a = number_of_slack_surplus_variables)
    number_of_artificial_variables_old = np.copy(a = number_of_artificial_variables)
    
    return tableau_current_size_old, number_of_constraints_old, number_of_slack_surplus_variables_old, number_of_artificial_variables_old

In [84]:
# This function creates cache arrays to reset the active arrays at the end of each recursion level
def create_cache_arrays_for_branch_and_bound_MILP_recursion(basic_variables, basic_feasible_solution, tableau_matrix):
    basic_variables_old = np.copy(a = basic_variables)

    basic_feasible_solution_old = np.copy(a = basic_feasible_solution)

    tableau_matrix_old = np.copy(a = tableau_matrix)
    
    return basic_variables_old, basic_feasible_solution_old, tableau_matrix_old

In [85]:
# This function handles the less than or greater than branches of the recursive tree of branch and bound
def branch_and_bound_MILP_recursive_less_or_greater_than(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index, last_variable_that_still_needs_to_become_integer_value, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, tableau_current_size_old, number_of_constraints_old, number_of_slack_surplus_variables_old, number_of_artificial_variables_old, basic_feasible_solution_old, basic_variables_old, tableau_matrix_old, new_constraint_inequality_direction, recursion_level):
    if new_constraint_inequality_direction == 1: # less than
        # Last_variable_that_still_needs_to_become_integer_or_binary_index <= new_constraint_constant
        # add_constraint
        error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size = \
            add_constraint(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, last_variable_that_still_needs_to_become_integer_or_binary_index, new_constraint_inequality_direction, np.floor(last_variable_that_still_needs_to_become_integer_value).astype(np.int64), recursion_level)
    else: # greater than
        # Last_variable_that_still_needs_to_become_integer_or_binary_index >= new_constraint_constant
        # add_constraint
        error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size = \
            add_constraint(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, last_variable_that_still_needs_to_become_integer_or_binary_index, new_constraint_inequality_direction, np.ceil(last_variable_that_still_needs_to_become_integer_value).astype(np.int64), recursion_level)
    
    if error_code == 0:
        # count_number_of_variables_needing_to_be_integer_or_binary_that_already_are
        number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive = \
            count_number_of_variables_needing_to_be_integer_or_binary_that_already_are(number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, basic_feasible_solution)

        if number_of_variables_needing_to_be_integer_or_binary_currently_integer_or_binary < number_of_variables_required_to_be_integer + number_of_variables_required_to_be_binary:
            # check_if_more_branch_and_bound_MILP_recursion_is_necessary
            error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = check_if_more_branch_and_bound_MILP_recursion_is_necessary(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, recursion_level)
        else:
            print_optimal_results(tableau_matrix[:, number_of_constraints, 0], basic_feasible_solution)

            # save_best_mixed_integer_optimal_variables_and_values
            best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
                save_best_mixed_integer_optimal_variables_and_values(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code)
    elif error_code == 1:
        print("branch_and_bound_MILP_recursive_less_or_greater_than: Infeasible!")
    elif error_code == 2:
        if best_milp_error_code == 1:
            best_milp_error_code = 2
        print("branch_and_bound_MILP_recursive_less_or_greater_than: Unbounded!")

    # reset_branch_and_bound_MILP_recursive_counts_and_arrays
    tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_feasible_solution, basic_variables, tableau_matrix = \
        reset_branch_and_bound_MILP_recursive_counts_and_arrays(tableau_current_size_old, number_of_constraints_old, number_of_variables, number_of_slack_surplus_variables_old, number_of_artificial_variables_old, basic_feasible_solution_old, basic_variables_old, tableau_matrix_old, basic_feasible_solution, basic_variables, tableau_matrix)
    
    return error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values

In [86]:
# This function checks if more recursion of MILP branch and bound is necessary
def check_if_more_branch_and_bound_MILP_recursion_is_necessary(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, recursion_level):
    if recursion_level < max_branch_and_bound_recursion_depth + 1:
        if maximization_problem == 1:
            if tableau_matrix[0, number_of_constraints, 0].astype(np.float64) / tableau_matrix[1, number_of_constraints, 0] > best_mixed_integer_optimal_value_double:
                # Keep going down the rabbit hole
                # branch_and_bound_MILP_recursive
                error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
                    branch_and_bound_MILP_recursive(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, recursion_level + 1)
        else:
            if -tableau_matrix[0, number_of_constraints, 0].astype(np.float64) / tableau_matrix[1, number_of_constraints, 0] < best_mixed_integer_optimal_value_double:
                # Keep going down the rabbit hole
                # branch_and_bound_MILP_recursive
                error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values = \
                    branch_and_bound_MILP_recursive(maximization_problem, number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, last_variable_that_still_needs_to_become_integer_or_binary_index_recursive, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code, recursion_level + 1)

    return error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size, best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values

In [87]:
# This function saves the best mixed integer optimal variables and values
def save_best_mixed_integer_optimal_variables_and_values(maximization_problem, number_of_constraints, number_of_variables, basic_feasible_solution, tableau_matrix, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values, best_milp_error_code):
    if maximization_problem == 1:
        if tableau_matrix[0, number_of_constraints, 0].astype(np.float64) / tableau_matrix[1, number_of_constraints, 0] > best_mixed_integer_optimal_value_double:
            best_milp_error_code = 0

            best_mixed_integer_optimal_value_double = tableau_matrix[0, number_of_constraints, 0].astype(np.float64) / tableau_matrix[1, number_of_constraints, 0]

            best_mixed_integer_optimal_value[:] = tableau_matrix[:, number_of_constraints, 0]

            best_mixed_integer_variable_values = basic_feasible_solution[:, 0:number_of_variables]
    else:
        if -tableau_matrix[0, number_of_constraints, 0].astype(np.float64) / tableau_matrix[1, number_of_constraints, 0] < best_mixed_integer_optimal_value_double:
            best_milp_error_code = 0

            best_mixed_integer_optimal_value_double = -tableau_matrix[0, number_of_constraints, 0].astype(np.float64) / tableau_matrix[1, number_of_constraints, 0]

            best_mixed_integer_optimal_value[0] = -tableau_matrix[0, number_of_constraints, 0]
            best_mixed_integer_optimal_value[1] = tableau_matrix[1, number_of_constraints, 0]

            best_mixed_integer_variable_values = basic_feasible_solution[:, 0:number_of_variables]
                
    return best_milp_error_code, best_mixed_integer_optimal_value_double, best_mixed_integer_optimal_value, best_mixed_integer_variable_values

In [88]:
# This function resets the counts and arrays during recursive branch and bound MILP
def reset_branch_and_bound_MILP_recursive_counts_and_arrays(tableau_current_size_old, number_of_constraints_old, number_of_variables, number_of_slack_surplus_variables_old, number_of_artificial_variables_old, basic_feasible_solution_old, basic_variables_old, tableau_matrix_old, basic_feasible_solution, basic_variables, tableau_matrix):
    # Reset counts
    tableau_current_size = np.copy(a = tableau_current_size_old)
    number_of_constraints = np.copy(a = number_of_constraints_old)
    number_of_slack_surplus_variables = np.copy(a = number_of_slack_surplus_variables_old)
    number_of_artificial_variables = np.copy(a = number_of_artificial_variables_old)

    # Reset arrays
    basic_feasible_solution = basic_feasible_solution_old[:, 0:number_of_variables + number_of_slack_surplus_variables_old + number_of_artificial_variables_old]

    basic_variables = basic_variables_old[0:number_of_constraints_old]

    tableau_matrix = tableau_matrix_old[:, 0:tableau_current_size_old[0], 0:tableau_current_size_old[1]]

    return tableau_current_size, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_feasible_solution, basic_variables, tableau_matrix

In [89]:
# This function adds a constraint to the previous optimal solution (warm-start)
def add_constraint(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size, last_variable_that_still_needs_to_become_integer_or_binary_index, new_constraint_inequality_direction, new_constraint_constant, recursion_level):
    if new_constraint_inequality_direction == 1: # if less than or equal to
        # Create new >= constraint for first variable that still needs to become an interger
        print("\nadd_constraint: New constraint is x{} <= {} -> -x{} >= {} at recursion level {}".format(last_variable_that_still_needs_to_become_integer_or_binary_index + 1, new_constraint_constant, last_variable_that_still_needs_to_become_integer_or_binary_index + 1, -new_constraint_constant, recursion_level))
    else: # if greater than or equal to
        # Create new >= constraint for first variable that still needs to become an interger
        print("\nadd_constraint: New constraint is x{} >= {} at recursion level {}".format(last_variable_that_still_needs_to_become_integer_or_binary_index + 1, new_constraint_constant, recursion_level))

    # Increase tableau size to make room for two new rows and columns due to the new constraint
    tableau_current_size += 2
    
    # Take contiguous slice from incoming tableau that will be carried over to new tableau
    original_tableau_matrix_slice = tableau_matrix[:, 0:tableau_current_size[0] - 3, 0:tableau_current_size[1] - 2]

    # Take the incoming tableau's objective function row
    original_tableau_matrix_objective_row = np.expand_dims(a = tableau_matrix[:, tableau_current_size[0] - 3, 0:tableau_current_size[1] - 2], axis = 1)

    # Create new artificial objective row for phase 1 simplex
    new_artificial_objective_row = np.stack(arrays = [np.zeros(shape = [1, tableau_current_size[1] - 2], dtype = np.int64), np.ones(shape = [1, tableau_current_size[1] - 2], dtype = np.int64)], axis = 0)

    # Create new constraint row
    new_constraint_constant_element = np.stack(arrays = [np.array(object = [[-new_constraint_inequality_direction * new_constraint_constant]], dtype = np.int64), np.ones(shape = [1, 1], dtype = np.int64)], axis = 0)

    variable_mask = np.arange(number_of_variables) == last_variable_that_still_needs_to_become_integer_or_binary_index

    new_constraint_variable_vector = np.stack(arrays = [np.expand_dims(a = np.where(variable_mask, -new_constraint_inequality_direction * np.ones(shape = [number_of_variables], dtype = np.int64), np.zeros(shape = [number_of_variables], dtype = np.int64)), axis = 0), np.ones(shape = [1, number_of_variables], dtype = np.int64)], axis = 0)

    old_constraints_slack_surplus_zero_vector = np.stack(arrays = [np.zeros(shape = [1, number_of_constraints], dtype = np.int64), np.ones(shape = [1, number_of_constraints], dtype = np.int64)], axis = 0)

    new_constraint_row = np.concatenate([new_constraint_constant_element, new_constraint_variable_vector, old_constraints_slack_surplus_zero_vector], axis = 2)
    
    basic_variable_mask = basic_variables[0:number_of_constraints] == last_variable_that_still_needs_to_become_integer_or_binary_index + 1
    
    basic_variable_index = np.squeeze(a = np.where(basic_variable_mask)[0])

    # Add variable's basic row to new constraint row, if less than or equal to
    true_array = np.stack(arrays = list(map(lambda j: rational_addition(new_constraint_row[0, 0, j], new_constraint_row[1, 0, j], original_tableau_matrix_slice[0, basic_variable_index, j], original_tableau_matrix_slice[1, basic_variable_index, j]), np.arange(tableau_current_size[1] - 2))), axis = 1)

    # Subtract variable's basic row from new constraint row, if greater than or equal to
    false_array = np.stack(arrays = list(map(lambda j: rational_addition(new_constraint_row[0, 0, j], new_constraint_row[1, 0, j], -original_tableau_matrix_slice[0, basic_variable_index, j], original_tableau_matrix_slice[1, basic_variable_index, j]), np.arange(tableau_current_size[1] - 2))), axis = 1)

    new_constraint_row_added_or_subtracted = np.expand_dims(a = np.where(new_constraint_inequality_direction == 1, true_array, false_array), axis = 1)

    # This is the full new tableau minus the last two columns, in other words tableau_matrix[:, 0:tableau_current_size[0], 0:tableau_current_size[1] - 2]
    tableau_old_columns = np.concatenate([original_tableau_matrix_slice, new_constraint_row_added_or_subtracted, original_tableau_matrix_objective_row, new_artificial_objective_row], axis = 1)

    # Fill in last two new columns for constraint's new surplus and artificial variables
    new_surplus_and_artificial_variable_columns_numerator = np.concatenate([np.zeros(shape = [number_of_constraints, 2], dtype = np.int64), np.array(object = [[-1, 1]], dtype = np.int64), np.zeros(shape = [1, 2], dtype = np.int64), np.array(object = [[0, -1]], dtype = np.int64)], axis = 0)

    new_surplus_and_artificial_variable_columns_denominator = np.ones(shape = [tableau_current_size[0], 2], dtype = np.int64)

    new_surplus_and_artificial_variable_columns = np.stack(arrays = [new_surplus_and_artificial_variable_columns_numerator, new_surplus_and_artificial_variable_columns_denominator], axis = 0)

    # Combine everything into the final new tableau matrix
    tableau_matrix = np.concatenate([tableau_old_columns, new_surplus_and_artificial_variable_columns], axis = 2)
    
    number_of_artificial_variables += 1

    # Update basic variables
    basic_variables = np.concatenate([basic_variables[0:number_of_constraints], np.array(object = [tableau_current_size[1] - 1], dtype = np.int64)])

    number_of_constraints += 1
    number_of_slack_surplus_variables += 1

    # Update the basic feasible solution
    basic_feasible_solution = realloc_basic_feasible_solution(basic_feasible_solution, number_of_variables + number_of_slack_surplus_variables + number_of_artificial_variables)
    
    basic_feasible_solution = update_basic_feasible_solution(tableau_current_size[1] - 1, number_of_constraints, basic_variables, basic_feasible_solution, tableau_matrix)

    # This function finds the optimal solution for the given variables and constraints
    # simplex_algorithm
    error_code, basic_variables, basic_feasible_solution, tableau_matrix, number_of_artificial_variables, tableau_current_size = simplex_algorithm(number_of_constraints, number_of_variables, number_of_slack_surplus_variables, number_of_artificial_variables, tableau_matrix, basic_variables, basic_feasible_solution, tableau_current_size)
    
    return error_code, number_of_constraints, number_of_slack_surplus_variables, number_of_artificial_variables, basic_variables, basic_feasible_solution, tableau_matrix, tableau_current_size

# Helper functions

In [90]:
# This function prints the optimal objective function value and the variable values
def print_optimal_results(optimal_objective_function_value, optimal_variable_values):
    print("print_optimal_results: optimal_objective_function_value = \n{}".format(optimal_objective_function_value))
    print("print_optimal_results: optimal_objective_function_value(float) = \n{}".format(optimal_objective_function_value[0].astype(np.float64) / optimal_objective_function_value[1]))
    
    print("print_optimal_results: optimal_variable_values = \n{}".format(optimal_variable_values))
    print("print_optimal_results: optimal_variable_values(float) = \n{}".format(optimal_variable_values[0, :].astype(np.float64) / optimal_variable_values[1, :]))
        
    return

In [91]:
# This function adds two columns to basic feasible solution array
def realloc_basic_feasible_solution(basic_feasible_solution, number_of_total_variables):
    additional_basic_feasible_solution_cols = np.stack(arrays = [np.zeros(shape = [2], dtype = np.int64), np.ones(shape = [2], dtype = np.int64)])

    basic_feasible_solution = np.concatenate([basic_feasible_solution[:,0:number_of_total_variables], additional_basic_feasible_solution_cols], axis = 1)
    
    return basic_feasible_solution

In [92]:
# This function adds long long rationals and keeps their numerators and denominators each as small as possible
def rational_addition(A_numerator, A_denominator, B_numerator, B_denominator):
    C_numerator = A_numerator * B_denominator + A_denominator * B_numerator
    C_denominator = A_denominator * B_denominator
    
    temp_gcd = greastest_common_denominator(C_numerator, C_denominator)
    C_numerator = C_numerator // temp_gcd
    C_denominator = C_denominator // temp_gcd
    
    return C_numerator, C_denominator

In [93]:
# This function divides long long rationals and keeps their numerators and denominators each as small as possible
def rational_division(A_numerator, A_denominator, B_numerator, B_denominator):
    C_numerator = A_numerator * B_denominator
    C_denominator = A_denominator * B_numerator
    
    temp_gcd = greastest_common_denominator(C_numerator, C_denominator)
    C_numerator = C_numerator // temp_gcd
    C_denominator = C_denominator // temp_gcd
    
    if C_denominator < 0: # if denominator is negative
        C_numerator = -C_numerator
        C_denominator = -C_denominator
        
    return C_numerator, C_denominator

In [94]:
# This function finds the greatest common denominator between a and b
def greastest_common_denominator(a, b):
    while b != 0:
        t = b
        b = a % b
        a = t
        
    return np.abs(a)

# MAIN

In [95]:
# This is the main function
def main_function():
    error_code = 0

    #*******************************************************************************
    #******************************** READ INPUTS **********************************
    #*******************************************************************************

    # Sizes
    number_of_constraints, number_of_variables = read_number_of_constraints_and_variables()

    # Variable special requirements
    variable_special_requirements, number_of_variables_required_to_be_standard, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, number_of_variables_required_to_be_unrestricted = \
        read_and_count_variable_special_requirements()

    # Optimization problem type
    maximization_problem = read_objective_function_maximization_or_minimization()

    # Objective function
    objective_function_initial_constant = read_objective_function_initial_constant()

    objective_function_coefficient_vector = read_objective_function_coefficient_vector()

    # Constraints
    constraint_inequality_direction_vector = read_constraint_inequality_direction_vector()

    constraint_constant_vector = read_constraint_constant_vector()

    constraint_coefficient_matrix = read_constraint_coefficient_matrix()

    #*******************************************************************************
    #******************************* OPTIMAL VALUES ********************************
    #*******************************************************************************

    # Optimums
    optimal_objective_function_value, optimal_variable_values = create_optimal_objective_function_and_variable_values(maximization_problem, number_of_variables)

    #*******************************************************************************
    #********************** MIXED INTEGER LINEAR PROGRAMMING ***********************
    #*******************************************************************************

    error_code, optimal_objective_function_value, optimal_variable_values = mixed_integer_linear_programming(maximization_problem, number_of_constraints, number_of_variables, number_of_variables_required_to_be_integer, number_of_variables_required_to_be_binary, variable_special_requirements, objective_function_initial_constant, objective_function_coefficient_vector, constraint_inequality_direction_vector, constraint_constant_vector, constraint_coefficient_matrix, optimal_objective_function_value, optimal_variable_values)

    if error_code == 0:
        print("\n\n************************************************************************************************************\n")
        print("************************************************************************************************************\n")
        print("*********************************************** FINAL SOLUTION ******************************************\n")
        print("************************************************************************************************************\n")
        print("************************************************************************************************************\n\n\n")

        print_optimal_results(optimal_objective_function_value, optimal_variable_values)
    elif error_code == 1:
        print("main_function: Problem is infeasible!")
    elif error_code == 2:
        print("main_function: Problem is unbounded!")
    else:
        print("main_function: How did I get an error_code = {} that is > 2?".format(error_code))

    return

In [96]:
main_function()

read_number_of_constraints_and_variables: number_of_constraints = 
4
read_number_of_constraints_and_variables: number_of_variables = 
2
read_and_count_variable_special_requirements: variable_special_requirements = 
[1 1]
read_and_count_variable_special_requirements: number_of_variables_required_to_be_standard = 
0
read_and_count_variable_special_requirements: number_of_variables_required_to_be_integer = 
2
read_and_count_variable_special_requirements: number_of_variables_required_to_be_binary = 
0
read_and_count_variable_special_requirements: number_of_variables_required_to_be_unrestricted = 
0
read_objective_function_maximization_or_minimization: maximization_problem = 
1
read_objective_function_initial_constant: objective_function_initial_constant = 
0.0
read_objective_function_coefficient_vector: objective_function_coefficient_vector = 
[ 1. -1.]
read_constraint_inequality_direction_vector: constraint_inequality_direction_vector = 
[-1  1 -1  1]
read_constraint_constant_vector: cons